In [503]:
import pandas as pd
import numpy as np
import ta
import time
from datetime import datetime
from binance import Client
from binancekeys import api_key, secret_key
from plot_utils import plot_results
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [504]:
client = Client(api_key, secret_key)

In [505]:
def getdata(symbol, interval, start_str, end_str):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                      interval,
                                                      start_str,
                                                      end_str))
    frame = frame.iloc[:,:6]
    cols = ['time',
            f'open',
            f'high',
            f'low',
            f'close',
            f'volume']
    frame.columns = cols
    
    return frame

In [506]:
def applytechnicals(df):
    df['%K'] = ta.momentum.stoch(df.high, df.low, df.close, window=8, smooth_window=3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.close, window=14)
    df['macd'] = ta.trend.macd_diff(df.close)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

In [507]:
coins = ['XMR']
merge = False

for coin in coins:
    symbol = f'{coin}USDT'
    interval = client.KLINE_INTERVAL_15MINUTE
    start_str = 'Jun 12, 2022'
    end_str = 'Jun 14, 2022'
     
    print(f'gathering {coin}...')    
    coin_df = getdata(symbol, interval, start_str, end_str)
    
    if merge:
        df = pd.merge(coin_df, all_coins_df, how='inner',on=['time'])
    else:
        df = coin_df
        merge = True
    
    time.sleep(0.5)        

gathering XMR...


In [508]:
df['time'] = [datetime.fromtimestamp(ts / 1000) for ts in df['time']]

In [509]:
for col in df.columns:
    if not 'time' in col:
        df[col] = df[col].astype(float)

In [510]:
df.head()

,time,open,high,low,close,volume
0,2022-06-12 01:00:00,166.7,166.9,165.6,166.4,342.802
1,2022-06-12 01:15:00,166.4,166.9,166.1,166.7,222.323
2,2022-06-12 01:30:00,166.7,167.0,166.2,167.0,156.220
3,2022-06-12 01:45:00,167.0,167.2,166.4,166.9,439.013
4,2022-06-12 02:00:00,166.8,166.8,165.0,165.1,248.930


In [511]:
applytechnicals(df)

In [512]:
df.shape

(160, 10)

In [513]:
df.head()

,time,open,high,low,close,volume,%K,%D,rsi,macd
0,2022-06-12 09:15:00,160.4,162.1,160.3,161.8,763.578,92.857143,89.625081,60.868053,0.957672
1,2022-06-12 09:30:00,161.9,162.6,161.5,161.7,804.178,80.000000,85.311355,60.415652,0.934013
2,2022-06-12 09:45:00,161.8,162.5,161.3,161.8,966.714,82.222222,85.026455,60.729978,0.882128
3,2022-06-12 10:00:00,161.9,162.5,161.2,161.8,387.055,82.222222,81.481481,60.729978,0.806878
4,2022-06-12 10:15:00,161.8,162.5,161.3,161.5,568.563,75.555556,80.000000,59.097248,0.699945


In [514]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l] > df1.low[i]):
            pividlow=0
        if(df1.high[l] < df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def Kpivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1['%K'][l] > df1['%K'][i]):
            pividlow=0
        if(df1['%K'][l] < df1['%K'][i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 


In [515]:
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,5,5), axis=1)

In [516]:
df['Kpivot'] = df.apply(lambda x: Kpivotid(df, x.name, 5, 5), axis=1)

In [517]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def Kpointpos(x):
    if x['Kpivot']==1:
        return x['%K']-1
    elif x['Kpivot']==2:
        return x['%K']+1
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['Kpointpos'] = df.apply(lambda row: Kpointpos(row), axis=1)

In [541]:
df_plot = df[0:60]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=df_plot.index,
                open=df_plot.open,
                high=df_plot.high,
                low=df_plot.low,
                close=df_plot.close), row=1, col=1)

fig.add_scatter(x=df_plot.index, y=df_plot['pointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot", row=1, col=1)

fig.append_trace(go.Scatter(
    x=df_plot.index,
    y=df_plot['%K'],
    line_color="Red"
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=df_plot.index,
    y=df_plot['%D'],
    line_color="Green"
), row=2, col=1)

fig.add_scatter(x=df_plot.index, y=df_plot['Kpointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot", row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [519]:
csv_filename = f'XMR_{start_str}_{end_str}_{interval}.csv'
df.to_csv(csv_filename, index=False)

In [520]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
        
    def gettrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx = dfx.append(mask, ignore_index=True)
            #dfx = pd.concat((dfx, mask), axis=0, ignore_index=True)
            
        return dfx.sum(axis=0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
        self.df['Buy'] = np.where((self.df.trigger) &
                                 (self.df['%K'].between(20, 80)) & (self.df['%D'].between(20, 80)) &
                                 (self.df.rsi > 50) & (self.df.macd > 0), 1, 0)
        

In [521]:
signals = Signals(df, 5)

In [522]:
signals.decide()

In [523]:
#all_coins_df
df[df.Buy == 1]

,time,open,high,low,close,volume,%K,%D,rsi,macd,pivot,Kpivot,pointpos,Kpointpos,trigger,Buy
8,2022-06-12 11:15:00,158.9,161.7,158.6,161.4,2608.277,70.000000,24.444444,56.673095,0.184472,1,0,158.599,NaN,1,1
9,2022-06-12 11:30:00,161.5,161.8,160.7,160.8,487.603,56.410256,42.136752,54.068803,0.157013,0,0,NaN,NaN,1,1
10,2022-06-12 11:45:00,160.7,160.8,160.0,160.1,603.748,38.461538,54.957265,51.117500,0.083858,0,0,NaN,NaN,1,1
11,2022-06-12 12:00:00,160.2,161.5,159.6,161.5,1399.490,74.358974,56.410256,56.259854,0.119723,0,0,NaN,NaN,1,1
12,2022-06-12 12:15:00,161.5,161.5,159.9,160.2,376.648,50.000000,54.273504,50.904762,0.049459,0,0,NaN,NaN,1,1
27,2022-06-12 16:00:00,164.0,164.4,163.0,163.3,1470.365,78.750000,79.988263,57.319295,0.092312,0,0,NaN,NaN,1,1


In [524]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append( [self.balance_unit, time, sell_price] )
        self.balance_unit = 'USDT'

In [525]:
# VIP level 0, paying fees with BNB = 0.00075%
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925)

coin = 'XMR'

open_position = False

# for each row where Buy == 1, buy @ open price and sell @ close price
for i in range(len(df[df.Buy == 1])):
    # BUY
    if not open_position:
        buyprice = float(df[f'{coin}-USD_Open'].iloc[i])

        target_price = float(buyprice * 1.005)
        stop_price = float(buyprice * 0.995)

        buytime = df['OpenTime'].iloc[i]
        quantity = env.balance_amount / buyprice

        env.buy(coin, buyprice, buytime)
        open_position = True

        buy_order = f'{buytime} : {env.balance_unit} : BUY : price = {buyprice} : QTY : {quantity}, Target : {target_price} , Stop : {stop_price}'   
        print(buy_order)
        print()
    
    # SELL
    close_price = (float)(df[f'{coin}-USD_Close'].iloc[i])

    if close_price <= stop_price or close_price >= target_price:
        sell_price = close_price
        selltime = df.OpenTime.iloc[i]
            
        env.sell(sell_price, selltime)
        open_position = False
        sell_order = f'{selltime} : {env.balance_unit} : SELL : price = {sell_price}'

        print(sell_order)
        print(f'Profit : {sell_price - buyprice}')
        print()

if open_position:
    env.sell(close_price, selltime)
    open_position = False
    sell_order = f'{selltime} : {env.balance_unit} : SELL : price = {close_price}'
            
    print(sell_order)
    print(f'Profit : {close_price - buyprice}')
    print()    

print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')    

KeyError: 'XMR-USD_Open'